In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# Mounting Google Drive and Setting up dataset from Kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import os

# changing the working directory
%cd /content/gdrive/My Drive/Normalized

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1NstkfsDWmtDt6AM0JqALnpB2dcIm3Z1u/Normalized


# Importing required Libraries

In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.3MB 42kB/s 


In [ ]:
!nvidia-smi

Sun Apr 18 10:00:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    33W /  70W |    222MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# importing required libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
train_path = "/content/gdrive/MyDrive/Normalized/train"
valid_path = "/content/gdrive/MyDrive/Normalized/test"

# Optimisation - Adam with learning rate of 0.05
opt = Adam(lr = 0.05)

# Image Augmentation
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path, target_size = (150, 150), batch_size = 16, class_mode = 'categorical')

Found 38478 images belonging to 2 classes.


In [ ]:
test_set = test_datagen.flow_from_directory(valid_path, target_size = (150, 150), batch_size = 16, class_mode = 'categorical' )

Found 9618 images belonging to 2 classes.


# Functions

In [ ]:
def build_model(model):
    for layer in model.layers:
        layer.trainable = False

    # add average global pooling layer, 1 fully connected layer, and a softmax classifier
    gap = GlobalAveragePooling2D()(model.output)
    d = Dropout(0.50)(gap)
    fcl = Dense(2, activation = 'softmax')(d)

    # define new model
    model = Model(inputs = model.inputs, outputs = fcl)

    return model

def cm_cr(model):
  global test_set, training_set
  test_set.reset()
  Y_pred = model.predict(test_set, len(test_set) // 16 + 1)
  y_pred = np.argmax(Y_pred, axis = 1)
  print("Confusion Matrix")
  print(confusion_matrix(test_set.classes, y_pred))
  
  print("Classification Report")
  target_names = list(training_set.class_indices.keys())
  print(classification_report(test_set.classes, y_pred, target_names = target_names))

def classify(image, model):
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  pred = model.predict(image)
  if pred[0][0] > pred[0][1]:
    print("0 --> Non IDC")
  else:
    print("1 --> IDC")

In [ ]:
%cd /content/gdrive/My Drive/savedModels

/content/gdrive/My Drive/savedModels


# ResNet50

In [ ]:
resnet = ResNet50(input_shape = (150, 150, 3), weights = 'imagenet', include_top = False)
resnet = build_model(resnet)
resnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 75, 75, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
# Checkpoint and Early Stopping
mcp_save = ModelCheckpoint('resnetlr0.05', save_best_only = True, monitor = 'val_accuracy', mode = 'max')
early_stopping_monitor = EarlyStopping(monitor = 'val_accuracy', mode = 'max', restore_best_weights = True, patience = 10)

In [ ]:
# RUNNING FIRST TIME
resnet.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])

h = resnet.fit(training_set, validation_data = test_set, epochs = 100, steps_per_epoch =len(training_set)//16 ,
               validation_steps = len(test_set)//16, callbacks=[mcp_save, early_stopping_monitor])

In [ ]:
# RUNNING SECOND TIME ONWARDS
resnet = load_model('resnetlr0.01')
for layer in resnet.layers[:-3]:
       layer.trainable = False
       
h = resnet.fit(training_set, validation_data = test_set, epochs = 100, steps_per_epoch =len(training_set)//32 ,
               validation_steps = len(test_set)//32, callbacks=[mcp_save, early_stopping_monitor])

# Confusion Matrix And Classification Report

In [ ]:
resnet.evaluate(test_set)

301/301 [==============================] - 816s 3s/step - loss: 0.6454 - accuracy: 0.6841


[0.6453955173492432, 0.6841338872909546]

In [ ]:
cm_cr(resnet)

Confusion Matrix
[[4171  638]
 [2400 2409]]
Classification Report
              precision    recall  f1-score   support

           0       0.63      0.87      0.73      4809
           1       0.79      0.50      0.61      4809

    accuracy                           0.68      9618
   macro avg       0.71      0.68      0.67      9618
weighted avg       0.71      0.68      0.67      9618

